# Causal inference applied to an electric circuit
This will be a case of causal inference applied to a simple electric circuit. Firstly, this electric circuit will be introduced. Then, it will be explained that non-experimental time series recordings should be augmented with engineering expertise to infer causalities. Finally, a causal effect will be inferred and while enabling fault detection and diagnostics on the fly. 

## Introduction to the case

<table>
    <thead>
        <tr>
            <th> <th>Time <th>V<sub>0</sub> <th> V<sub>1</sub> <th> S<sub>1</sub>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=11> <img src="figures/CausalInference01.png" width="350"/>  <td>[  0:  4] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[  5:14] <td> 3,4 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[15:18] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[19:23] <td> 3,4 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[24:30] <td> 0,0 <td> 0,1 <td> 0
        </tr>
        <tr>
            <td>[31:36] <td> 3,5 <td> 1,4 <td> 1
        </tr>
        <tr>
            <td>[37:38] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[39:40] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[40:49] <td> 2,5 <td> 2,5 <td> 1
        </tr>
        <tr>
            <td>[50:53] <td> 3,5 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[54:60] <td> 0,0 <td> 0,0 <td> 0
        </tr>
    </tbody>    
</table>

The electric circuit in Figure 1 consists of a light, two resistors, a ground connection and a switch. The dashed area shows the time series recordings: 
- the position of the switch (S<sub>1</sub>), 
- the voltage V<sub>0</sub> before the light,
- the voltage V<sub>1</sub> after the light.

These time series just came from the normal course of operations. So, they have been collected by non-experimental research. Click [here](https://nbviewer.jupyter.org/github/chrisrijsdijk/RAMS/blob/master/notebook/Arduino_3Vars/CausalInferenceSitetextCaseData.ipynb) to see the script that generates these time series in np.arrays.

## Inferences of statistical dependencies from the time series

![image.png](figures/CausalInference02.png)

Figure 2 shows that the time series data cluster very well which means that there are strong dependencies between them. For example, the mean of the voltage $\overline{V}_{0}$ given an open switch $S_{1}=0$ is given by:

$  [\overline{V}_{0}|S_{1}=0]=0,0V $

The mean of the voltage $V_{0}$ given an open switch $S_{1}=1$ is given by:

$ [\overline{V}_{0}|S_{1}=1]=3,2V $

As the individual data points of the Voltage $V_{0}$ given some switch position $S_{1}$ can be entirely separated, one concludes that these variables are strongly dependent within this sample. However, the causality of this dependency remains undetermined. The dependency may be explained by $V_{0} \rightarrow S_{1}$ or by $V_{0} \leftarrow S_{1}$ or by $V_{0} \leftarrow X \rightarrow S_{1}$. Causality does not just follow from nonexperimental recordings, rather it requires additional knowledge. Still, Figure 2 indicates that the time series are best summarised by three clusters whereas the switch $S_{1}$ only has two positions. Therefore, the switch $S_{1}$ alone cannot entirely explain the voltages. To be more precise, given $S_{1}=1$ the voltages tend to two clusters.

## Positing causalities using engineering expertise

Technical systems are typically designed to have some effect. In this case, the light should listen to the switch to provide the desired effect. So, the circuit has been designed to realise the causality $V_{0} \leftarrow S_{1}$. The engineer may even use a topdown Failure Mode and Effect Analysis (FMEA) to identify the variables (failure modes) that perturb the required causality $V_{0} \leftarrow S_{1}$. Let the Table survey all failure modes that have been deemed possible. These failure modes are seen as dichotomous variables that are either absent or present.

<table>
    <tr>
        <th>Component <th>Failure mode <th>Code <th>Failure effect
    </tr>
    <tr>
        <td>wire to light <td>break circuit <td>F<sub>1</sub> <td>No light when switched on
    </tr>
    <tr>
        <td>light <td>break circuit <br /> short circuit <td>F<sub>2</sub> <br />F<sub>3</sub> <td>No light when switched on
    </tr>
    <tr>
        <td>wire to ground <td>break circuit <td>F<sub>4</sub> <td>No light when switched on
    </tr>
</table>

Evidently, this table is arbitrary and operating experience may reveal that this bottom-up FMEA appears to be incomplete. In a conventional FMEA, the effects of the failure modes will involve HSE, operational and non operational impact. However, for this exercise in causal inference, the expected sensor values, given *one* particular failure mode are important. The table below reflects engineering judgement and these values are supposed to be *calculated* at the stage of design. Then, there is no opportunity to involve time series recordings.
 
<table>
    <tr>
        <th>Component <th>Failure mode <th>Code <th colspan=2>Switch closed S<sub>1</sub>=1 <th colspan=2>Switch open S<sub>1</sub>=0
    </tr>
    <tr>
        <td> <td> <td> <td colspan=1>V<sub>0</sub> in V<td colspan=1>V<sub>1</sub> in V<td colspan=1>V<sub>0</sub> in V<td colspan=1>V<sub>1</sub> in V
    </tr>
    <tr>
        <td> <td>HEALTHY SYSTEM <td> <td colspan=1>3.4 <td colspan=1>1.4 <td colspan=1>0.0 <td colspan=1>0.0
    </tr>
    <tr>
        <td>wire to light <td>break circuit <td>F<sub>1</sub> <td colspan=1>0.0 <td colspan=1>0.0 <td colspan=1>0.0 <td colspan=1>0.0 
    </tr>
    <tr>
        <td>light <td>break circuit <br /> short circuit <td>F<sub>2</sub> <br />F<sub>3</sub> <td colspan=1>5.0 <br />2.5 <td colspan=1>0.0 <br />2.5 <td colspan=1>0.0 <br />0.0 <td colspan=1>0.0 <br />0.0 
    </tr>
    <tr>
        <td>wire to ground <td>break circuit <td>F<sub>4</sub> <td colspan=1>5.0 <td colspan=1>5.0 <td colspan=1>0.0 <td colspan=1>0.0 
    </tr>
</table>

This Table is thought to collide on the voltages, i.e. $S_{1} \rightarrow V_{x} \leftarrow F_{y}$. So, the switch $S_{1}$ and the failure modes $F_{y}$ together cause the voltages $V_{x}$. To reveal the Average Conditional Treatment Effect (ATE) of the switch $S_{1}$ on the voltage $V_{0}$ is given by:
    
$E[V_{0}(S_{1})|F_{y}]=E[V_{0}(1)|F_{y}]-E[V_{0}(0)|F_{y}] $

By summing out the effect of all failure modes $F_{y}$, the unconditional average treatment effect $E[V_{0}(S_{1})]$ follows:

$E[V_{0}(S_{1})]= E_{F_{y}}[ E[V_{0}(1)|F_{y}]-E[V_{0}(0)|F_{y}] ] $

To assess the unconditional average treatment effect, each system state should be assessed. As both the switch $S_{1}$ and the failure modes $F_{y}$ are dichotomous, there are $2^5=32$ system states for which the engineer should assess the expectation of the voltages $E[V_{x}]$. As the FMEA table above only specifies the voltages for $8$ system states, the unconditional treatment effect cannot be assessed. Here, the system states that include multiple failure modes are lacking. In the practice of reliability engineering, an FMEA analysis has often been deemed as labour intensive and various practices to lower the analysis burden by filtering out the more important system states upfront have been implemented. It should be realised that an average treatment effect assessment is encumbered with an even larger analysis burden. However, it will be shown that an assessment of a conditional average treatment effect is already valuable for fault detection.

## Engineering judgement augmented with recordings

This section just integrates the findings from the time series recordings and the causalities deduced by engineering expertise. From the time series, we have been able to calculate the mean voltages conditional on the switch $[\overline{V}_{x}|S_{1}]$ but these means do not represent the average treatment effect $E[V_{x}(S_{1})]$. As the time series of the failure modes $F_{y}$ are missing, these time series may be heavily biased towards a few system states that were predominant during this specific operation. Evidently, the time series even do not cover all $8$ system states from the FMEA table. For example the voltages $(V_{0}=5.0,V_{1}=0.0)$ caused by $(S_{1}=1,F_{1}=0,F_{2}=1,F_{3}=0,F_{4}=0)$ are not in the time series which means that this system state is missing. The number of observed system states may be delimited by (i) the non-experimental research construct or by (ii) laws and regulations. In fact, there are $32$ rather than $8$ possible system states. In practice, it is not uncommon that (i) engineers only specify a limited number of system states and that (ii) the recordings are incomplete. Therefore, the unconditional average treatment effect is *not* assessable and neither observable. The good news however is that 

*a conditional average treatment effect is often very useful for fault detection and diagnostics.*

For example, the time series show two clusters at a closed switch $(S_{1}=1$, namely $(V_{0}=3.4,V_{1}=1.5)$ and $(V_{0}=2.5,V_{1}=2.5)$. The FMEA Table indicates that these clusters correspond with particular system states, namely $(S_{1}=1,F_{1}=0,F_{2}=0,F_{3}=0,F_{4}=0)$ and $(S_{1}=1,F_{1}=0,F_{2}=0,F_{3}=1,F_{4}=0)$ respectively. Knowledge of the time series then provides information about the applicable system state. Once the switch is open $(S_{1}=0$, many system states might have been responsible for the voltages $(V_{0}=0.0,V_{1}=0.0)$ which hampers fault detection and diagnostics. The applicable system state is therefore uncertainty. The good news however is that

*uncertainty about the applicable system state is often not equal to universal doubt*

In practice, the time series may be enriched straightforwardly with operating experience. For example, if some open switch time interval $(S_{1}=0$ appeared to be maintenance free and that the neighbouring a closed switch $(S_{1}=1$ time intervals were observably healthy, it may not be very bold to assume that the system state during this open switch interval was also in fact healthy. Although almost any failure mode $F_{y}$ is equally possible at an open switch, the enriched time series may well indicate the most likely system state. In this way, the enriched time series may support fault detection and diagnostics again at some enhanced risk. Sight on the applicable system state may also indicate the actual usage of the system. Sight on the usage may be informative assess load and health which are more direct causes of a failure mode. Eventually, predictive maintenance may be supported in this way.

# [Click here to see the script](https://nbviewer.jupyter.org/github/chrisrijsdijk/RAMS/blob/master/notebook/Arduino_3Vars/Arduino_diagnostics_ensemble3VarsVal.ipynb) 
